In [ ]:
import sys
import os

import plotly.express as px
import plotly.offline as pyo

import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"



In [ ]:
from datetime import datetime
import data_config

# read processed sample data
sampled = pd.read_pickle(data_config.current_sample_sf)
print(sampled.columns)

color_map_l = data_config.color_map_l
print(color_map_l)

sampled['wk'] = sampled['created_at'].apply(lambda x: datetime.strptime(x, '%a %b %d %H:%M:%S %z %Y').isocalendar()[1])
print(set(sampled['wk']))

In [ ]:
def plot_frame(data, color_map, color_column="time_color",):
    pyo.init_notebook_mode()
    
    fig = px.scatter(
        data,
        x="x",
        y="y",
        color=color_column,
        color_discrete_map=color_map,
        opacity=1,
        custom_data=['hover_text', 'favorite_count', 'retweet_count', 'id', 'label', 'verified'],
        size="size",
#         symbol='verified', 
#         symbol_sequence=['circle','x'],
#         title="Semantic Clustering of Monkeypox Discussion on Twitter",
    )

    fig.layout.showlegend = False

    fig.update_traces(
        hovertemplate='%{customdata[0]}<br><br>'
            +'Likes: %{customdata[1]}   Retweets: %{customdata[2]}<br>'
            +'Label: %{customdata[5]}<br>'
            +'TweetID: %{customdata[4]}<br>'
            +'Verified User: %{customdata[5]}'
    )

    fig.layout.showlegend = False

    fig.update_layout(
        autosize=False,
        width=900,
        height=900,
    )
    
    fig.update_layout({
        'plot_bgcolor': 'rgba(255, 255, 255, 1)',
        'paper_bgcolor': 'rgba(255, 255, 255, 1)',
    })

    fig.update(layout_coloraxis_showscale=False)
    fig.update_yaxes(title='y', visible=False)
    fig.update_xaxes(title='x', visible=False)

    fig.show()
    
    return fig

## Mapping over time

In [ ]:
categories = set(sampled['k_means_category'])
col = px.colors.qualitative.Plotly[:len(categories)]
gray = '#BAB0AC'

for wk in [18, 22, 26, 29]:
    on = dict(zip(["{}_{}".format(wk, cat) for cat in categories],  col))
    for wk_off in set(sampled['wk']) - {wk}:
        off = {"{}_{}".format(wk_off, cat):gray for cat in categories}
        on.update(off)
    print(wk)
    plot_frame(sampled, on)

In [ ]:
set(sampled['wk'])

## Cumulative mapping over time

In [ ]:
# Generate time mapping with inactive points set as gray
categories = set(sampled['k_means_category'])
col = px.colors.qualitative.Plotly[:len(categories)]
gray = '#BAB0AC'

on = {}
for wk_off in set(sampled['wk']):
    off = {"{}_{}".format(wk_off, cat):gray for cat in categories}
    on.update(off)

    
for wk in set(sampled['wk']):
    new = dict(zip(["{}_{}".format(wk, cat) for cat in categories],  col))
    on.update(new)
    
    if wk in [20, 23, 26, 28, 29]:
        print(wk)
        print(datetime.strptime('2022 ' + str(wk) + ' 1', '%G %V %u'))
        plot_frame(sampled, on)

In [ ]:
for wk in [20, 23, 25, 27]:
    print(datetime.strptime('2022 ' + str(wk) + ' 7', '%G %V %u'))
    fig = plot_frame(sampled[sampled['wk'] <= wk], color_map_l, color_column='label')
    fig.write_image("../results/final/wk_{}.svg".format(wk))

In [ ]:
for wk in set(sampled['wk']):
        print(wk)
        print(datetime.strptime('2022 ' + str(wk) + ' 7', '%G %V %u').strftime("%B %d, %Y"))